<a href="https://colab.research.google.com/github/redman157/phamson/blob/master/Titanic_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pson2809","key":"c5bc723b596e5578b62accddd07da25f"}'}

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Nov 20 01:05 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
# List available datasets.
!kaggle datasets list

ref                                                      title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
kaggle/kaggle-survey-2018                                2018 Kaggle ML & DS Survey Challenge                 4MB  2018-11-03 22:35:07           3814  
lava18/google-play-store-apps                            Google Play Store Apps                               2MB  2018-09-18 20:49:49          20680  
olistbr/brazilian-ecommerce                              Brazilian E-Commerce Public Dataset by Olist        42MB  2018-10-31 16:15:27           4445  
jenny18/honey-bee-annotated-images                       The BeeImage Dataset: Annotated Honey Bee Images    50MB  2018-09-16 23:42:12           1707  
census/census-bureau-usa                                 United States Census           

In [7]:
# Copy the stackoverflow data set locally.
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 26.6MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.5MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.72MB/s]


In [8]:
X_train.head()

NameError: ignored

In [63]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender =  pd.read_csv('gender_submission.csv')
def loss(x,y):
  return np.add(np.square(x-y),2)
  return 1/2*m
def optimize(x,y,lr):
  #return theta += theta - np.multiply(loss,lr)
  pass

X = training
y = training.Survived

feature_train = X.drop(['Name','PassengerId'],axis = 1)
gioitinh= {'male':0,'female':1}
feature_train.Sex.replace(gioitinh,inplace = True)

embarked = {'S':0,'C':1,'Q':2}
feature_train.Embarked.replace(embarked,inplace= True)
feature_train.head()

feature_train.drop("Cabin",axis = 1)

feature_train['Family'] = feature_train.SibSp  + feature_train.Parch
feature_train.Family.loc[feature_train.Family > 0] =1 
feature_train.Family.loc[feature_train.Family == 0] = 0 

feature_train.Age.fillna(0,inplace = True)



X = feature_train.drop(['Ticket','SibSp','Parch','Cabin','Survived'],axis= 1)
print(X)
X_train,X_val,y_train,y_val = train_test_split(X,y)
model= DecisionTreeClassifier()
model.fit(X_train,y_train)
model_1 = RandomForestClassifier()
model_1.fit(X_train,y_train)


     Pclass  Sex   Age      Fare  Embarked  Family
0         3    0  22.0    7.2500       0.0       1
1         1    1  38.0   71.2833       1.0       1
2         3    1  26.0    7.9250       0.0       0
3         1    1  35.0   53.1000       0.0       1
4         3    0  35.0    8.0500       0.0       0
5         3    0   0.0    8.4583       2.0       0
6         1    0  54.0   51.8625       0.0       0
7         3    0   2.0   21.0750       0.0       1
8         3    1  27.0   11.1333       0.0       1
9         2    1  14.0   30.0708       1.0       1
10        3    1   4.0   16.7000       0.0       1
11        1    1  58.0   26.5500       0.0       0
12        3    0  20.0    8.0500       0.0       0
13        3    0  39.0   31.2750       0.0       1
14        3    1  14.0    7.8542       0.0       0
15        2    1  55.0   16.0000       0.0       0
16        3    0   2.0   29.1250       2.0       1
17        2    0   0.0   13.0000       0.0       0
18        3    1  31.0   18.000

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


ValueError: ignored

In [0]:
train_data('Family','Survived',X_train)
train_data('Pclass','Survived',X_train)
train_data('Sex','Survived',X_train)
train_data('Embarked','Survived',X_train)
train_data('Categor_Fare','Survived',X_train)
train_data('Name','Survived',X_train)

   Family  Survived
0   large  0.148936
1  medium  0.545455
2  single  0.303538
3   small  0.562738
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
   Sex  Survived
0    0  0.188908
1    1  0.742038
  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.336957
      Categor_Fare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081
      Name  Survived
0   Master  0.575000
1     Miss  0.697802
2       Mr  0.156673
3      Mrs  0.792000
4     Rare  0.444444
